# Objectives
Today : 27 11 2024  
Fetch data from the realt-time source from Jedha  
Run e set of tests with evidently to see if there are some change with the data from last month

## My problem
See 01_client_predict.py  
Models 3 & 4 from last month are inline  
Models 6 & 7 from yesterday are inline  
Models 4 & 7 never agree  




In [ ]:
import json
import time
import requests
import pandas as pd
from pathlib import Path
from datetime import datetime, timezone

k_DATA_PRODUCER = "https://real-time-payments-api.herokuapp.com/current-transactions"

(Path.cwd()/"data").mkdir(parents=True, exist_ok=True)


In [4]:


# -----------------------------------------------------------------------------
def get_one_transaction() -> pd.DataFrame:

    response = requests.get(k_DATA_PRODUCER)
    data = response.json()

    if isinstance(data, str):
        data = json.loads(data)

    columns = data["columns"]
    index = data["index"]
    rows = data["data"]

    df = pd.DataFrame(data=rows, index=index, columns=columns)

    # ! DANGER - 17 is hard coded
    col = df["current_time"]
    df.insert(17, "unix_time", col)

    # convert to date string
    df.rename(columns={"current_time": "trans_date_trans_time"}, inplace=True)
    timestamp = df["trans_date_trans_time"].iloc[0]
    date = datetime.fromtimestamp(timestamp / 1000, tz=timezone.utc)
    str_date = date.strftime("%Y-%m-%d %H:%M:%S")

    df["trans_date_trans_time"] = df["trans_date_trans_time"].astype(str)
    df.at[index[0], "trans_date_trans_time"] = str_date

    cols = df.columns.tolist()
    reordered_cols = [cols[-1]] + cols[:-1]  # the last col then all the other until the before last col
    df = df[reordered_cols]

    return df





In [5]:
def get_one_transaction_mock():
    return pd.DataFrame({'id': [1], 'value': [42], 'timestamp': [pd.Timestamp.now()]})

In [ ]:
# get_1_transaction = get_one_transaction_mock
get_1_transaction = get_one_transaction

# Define the output CSV filename relative to the notebook's directory
csv_filename = "production.csv"
output_path = Path.cwd()/"data"/csv_filename

# Create the file with headers if it doesn't exist
if not output_path.exists():
    # Open the file in write mode and write headers
    with output_path.open(mode='w') as f:
        # Write the headers (adjust based on your DataFrame structure)
        # headers = get_one_transaction().columns
        headers = get_1_transaction().columns
        f.write(','.join(headers) + '\n')

try:
    for i in range(500):  
        transaction = get_1_transaction()
        transaction.to_csv(output_path, mode='a', index=False, header=False)
        time.sleep(1)  
        print(f"Record {i} saved.")
except KeyboardInterrupt:
    # Seems not working
    print("Process interrupted by the user.")
except Exception as e:
    # Log any other unexpected errors
    print(f"An error occurred: {e}")
finally:
    print(f"Data saved to: {output_path}")


Record 0 saved.
Record 1 saved.
Record 2 saved.
Record 3 saved.
Record 4 saved.
Record 5 saved.
Record 6 saved.
Record 7 saved.
Record 8 saved.
Record 9 saved.
